In [1]:
##########
import pandas as pd
import numpy as np
import matplotlib as plt
import ast
import datetime
from ast import literal_eval
import sklearn
import json


In [ ]:
#####metadata preparation des donnees

In [2]:
metadata=pd.read_csv('movies_metadata.csv', delimiter = ',')


#1 supprimer les valeurs missing

metadata=metadata.dropna(subset=['id'])
metadata=metadata.dropna(subset=['title'])


#2 selectionner les film=released et transformer la variable adult en variable quali et supprimer les doublons id


#status released
metadata=metadata.loc[metadata['status']== 'Released']

#encode adult var
metadata1=pd.get_dummies(metadata, columns=["adult"])



#drop duplicates  

metadata2=metadata1.drop_duplicates()
metadata2=metadata2.drop_duplicates(subset='id', keep="first")


#selection variables


metadata2=metadata2[['genres', 
                     'id',
                     'original_language', 
                     'production_companies', 
                     'production_countries', 
                     'release_date',
                     'title'  , 
                     'vote_average',
                     'vote_count',
                     'adult_False', 
                     'adult_True'      ]]

/home/fitec/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
metadata2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44985 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                44985 non-null  object 
 1   id                    44985 non-null  object 
 2   original_language     44975 non-null  object 
 3   production_companies  44985 non-null  object 
 4   production_countries  44985 non-null  object 
 5   release_date          44907 non-null  object 
 6   title                 44985 non-null  object 
 7   vote_average          44985 non-null  float64
 8   vote_count            44985 non-null  float64
 9   adult_False           44985 non-null  uint8  
 10  adult_True            44985 non-null  uint8  
dtypes: float64(2), object(7), uint8(2)
memory usage: 3.5+ MB


In [4]:
#####  listes des categories des variables genres, production_companies et production_countries


#metadata2['title'].value_counts(dropna=False)


#fonction pr obtenir une liste des categories (max 20 elements a priori) a partir des dictionnaires



def categorie (data, variable):

    data['genrestest'] = data[variable].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    liste=data['genrestest'].str[0].value_counts()[:20].index.tolist()
    return liste


liste_genre=categorie(metadata2, 'genres')
liste_pcomp=categorie(metadata2, 'production_companies')
liste_pcount=categorie(metadata2, 'production_countries')

metadata2=metadata2.drop(['genrestest'], axis=1)





#########################fonction pour attribuer 1 a chaque element qui existe dans la liste des categories############################





def encoding_dic(data, variable, liste):
    
    #Colonne concernée
    #serie_col = data["genres"]
        
    #Liste des catégories conservées
    #liste = ['Drama', 'Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary', 'null']
    
    serie_col = data[variable]
    #Création de la colonne total : liste des catégories appartenant à la liste pour chaque ligne
    def add(x, liste_col):
        total = []
        if type(x) == str and x[0] == "[":
            a = ast.literal_eval(x)
            if len(a) > 0:
                for j in range(len(a)):
                    comp = a[j]["name"]
                    if comp in liste_col:
                        total.append(comp)
                if len(total) == 0:
                    total.append("null")
            else:
                total.append("null")
        return total
    
    total = serie_col.apply(lambda x : add(x, liste_col = liste))
    df = serie_col.to_frame()
    df["total"] = total
    
    #Création des colonnes pour le OneHotEncoding
    for genre in liste:
        df[genre] = 0
    
    #Complétion des colonnes OneHotEncoding grâce à la colonne total
    def add2(x,genre_cherche):
        for genre in x["total"]:
            if genre == genre_cherche:
                return 1
        return 0
    
    for genre in liste:
        df[genre] = df.apply(lambda x : add2(x, genre_cherche = genre), axis=1)
    
    return df



##########################################################""

dfgenres = encoding_dic(data=metadata2, variable="genres", liste=liste_genre)
dfgenres=dfgenres.drop(['genres', 'total'], axis=1)
dfprodcomp = encoding_dic(data=metadata2, variable="production_companies", liste=liste_pcomp)
dfprodcomp=dfprodcomp.drop(['production_companies', 'total'], axis=1)
dfprodcount = encoding_dic(data=metadata2, variable="production_countries", liste=liste_pcount)
dfprodcount=dfprodcount.drop(['production_countries', 'total'], axis=1)



#append ou concat moviedataset original avc les 3 dataframes dfgenres, dfprodcomp et  dfprodcount


#datamovienew=metadata2.append(dfgenres, ignore_index=True)
#datamovienew1=datamovienew.append(dfprodcomp,ignore_index=True )
#datamovienew2=datamovienew1.append(dfprodcount,ignore_index=True )

#datamovienew=pd.concat([metadata2, dfgenres, dfprodcomp, dfprodcount], axis=1)

datamovienew=pd.concat([metadata2, dfgenres, dfprodcomp, dfprodcount], axis=1)





datamovienew2=datamovienew.drop(['genres', 'production_companies', 'production_countries'], axis=1)




#####keywords




keywords=pd.read_csv('keywords.csv', delimiter = ',')
keywords=keywords.dropna(subset=['id'])
keywords=keywords.drop_duplicates(subset='id', keep="first")

liste_key=categorie(keywords, 'keywords')

keywords=keywords.drop(['genrestest'], axis=1)
dfkey= encoding_dic(data=keywords, variable="keywords", liste=liste_key)
dfkey=dfkey.drop(['keywords', 'total'], axis=1)
#datakey=keywords.append(dfkey, ignore_index=True)

datakey=pd.concat([keywords, dfkey], axis=1)

datakey=datakey.drop(['keywords'], axis=1)

####merge entre keywords et datamovienew2   

##datakey['id'] = datakey['id'].str.replace('.0', '')

datakey['id']=datakey['id'].astype(str)
datamovienew3=pd.merge(datamovienew2,datakey, on='id')

#test

#datamovienew3[[ 'woman director']].value_counts()
#datamovienew3.loc[datamovienew3['woman director']==1]
#datakey.loc[datakey['woman director']==1]


## import de ratings.csv et merge avec datamovienew3



ratings=pd.read_csv('ratings.csv', delimiter = ',')
ratings.dropna(subset=['userId'])
ratings=ratings.dropna(subset=['movieId'])
ratings=ratings.drop_duplicates()
ratings['movieId']=ratings['movieId'].astype('str')
ratings['userId']=ratings['userId'].astype('str')

df_movie = ratings.merge(datamovienew3, left_on='movieId', right_on='id', how='inner')


#df_movie[[ 'id', 'movieId', 'title']].head(5)   !!!!!
####FIN

